In [2]:
%run -i utils.py
%run -i preprocessing.py

ModuleNotFoundError: No module named 'pickle5'

# Hashtag Analysis

Dict of unique hashtag without 'redflag'

In [15]:
# unique_hashtags = {}

# for idx, row in tweets_filtered.iterrows():
#     hashtag_list = []
#     for hashtag in row["Hashtags"]:
#         hashtag = hashtag.lower()
#         if hashtag in hashtags_list_list:
#             unique_hashtags.setdefault("#"+hashtag, 0)
#             unique_hashtags['#'+hashtag] += 1
#             hashtag_list.append(hashtag)
#             tweets_filtered.at[idx, "hashtags"] = hashtag_list

In [1]:
uh_df = pd.DataFrame.from_dict(unique_hashtags, orient='index').reset_index()
uh_df.rename(columns = {'index':'Hashtag', 0:'Count'}, inplace=True)
uh_df.sort_values(by=['Count'], ascending=False, inplace=True)

NameError: name 'pd' is not defined

## Network Creation

In [ ]:
#uh = unique_hashtags.keys()
#uw = unique_words.keys()

### Hashtags Network

In [20]:
# Variable for filtering
h = 0

network = {} 
network_key = 0
for index, row in tweets_filtered.iterrows():
    #hashtags extracted from Tweet do not have the # sign in front of them but we will add it to differentiate hashtags from words
    combined_list = ['#'+hashtag for hashtag in row["hashtags"] if '#'+hashtag in unique_hashtags]
    #itertool product creates Cartesian product of each element in the combined list (hastag + words, both UNIQUE)
    for pair in itertools.product(combined_list, combined_list):
        #exclude self-loops and count each pair only once because our graph is undirected and we do not take self-loops into account
        if pair[0]!=pair[1] and not(pair[::-1] in network):
            network.setdefault(pair, 0)
            network[pair] += 1 

network = {key: value for (key, value) in network.items() if value >= h} #filter the network
network_df = pd.DataFrame.from_dict(network, orient="index")

network_df.reset_index(inplace=True) 
network_df.columns = ["pair","weight"] # undirected graph 
network_df.sort_values(by="weight", inplace=True, ascending=False) 

In [22]:
#to get weighted graph we need a list of 3-element tuplels (u,v,w) where u and v are nodes and w is a number representing weight
up_weighted = []
for edge in network:
    #we can filter edges by weight by uncommenting the next line and setting desired weight threshold
    #if(network[edge])>1: 
    up_weighted.append((edge[0],edge[1],network[edge]))

G = nx.Graph()
G.add_weighted_edges_from(up_weighted)

hashtag_nodes = uh_df.copy()
hashtag_nodes = hashtag_nodes.drop(hashtag_nodes[hashtag_nodes['Hashtag'] == '#redflag'].index)
hashtag_nodes["Label"] = hashtag_nodes["Hashtag"]
hashtag_nodes.rename(columns={"Hashtag":"Id"},inplace=True)
hashtag_nodes = hashtag_nodes.drop(columns=['Count'])

Number of nodes and number of unique_hashtags are different due to filter procedure. We only keep nodes.

In [25]:
from cdlib import algorithms, viz
coms = algorithms.louvain(G, weight='weight', resolution=1., randomize=False)
#coms = algorithms.surprise_communities(G)
#coms = algorithms.leiden(G)
#coms = algorithms.walktrap(G)

In [26]:
hashtag_nodes['Modularity Class'] = 0

k = 1
for community in coms.communities:
    for hashtag in hashtag_nodes['Label']:
        if hashtag in community:
            idx = hashtag_nodes[hashtag_nodes['Label'] == hashtag].index
            hashtag_nodes['Modularity Class'][idx] = k
    k += 1

hashtag_nodes = hashtag_nodes.drop(hashtag_nodes[hashtag_nodes['Modularity Class'] == 0].index) # this is for remove nodes without edges

D:\anaconda3\envs\network\lib\site-packages\ipykernel_launcher.py:8: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  


Node_list and Edge_list saving procedure for Gephi visualization

In [30]:
algo = '3'
# SAVE NODE LIST
hashtag_nodes.to_csv("./DATA_COMM/user/#nodelist_community_" + algo +".csv", index=False)

# We need to insert a column called "Modularity Class" and label
file = pd.read_csv("./DATA_COMM/user/#nodelist_community_" + algo +".csv", sep = ",", header = 0, names = ['Id','Modularity Class'])
file.insert(loc = 1, column = 'Label', value = file['Id'])
file.to_csv("./DATA_COMM/user/#nodelist_community_" + algo +".csv", index=False)

nx.write_weighted_edgelist(G, "./DATA_COMM/user/#edgelist_community_" + algo +".csv")

# We need to insert source, target, weight and type of edge
file = pd.read_csv("./DATA_COMM/user/#edgelist_community_" + algo +".csv", sep = " ", header = 0, names = ['Source','Target','Weight'])
file.insert(loc = 2, column = 'Type', value = ['undirected']*len(file))
file.to_csv("./DATA_COMM/user/#edgelist_community_" + algo +".csv", index=False)

## Filtering by hashtags
### Let's divide in community. In this case we choose the first one.
Each modularity class is a community

In [94]:
# hashtags_list = list(hashtag_nodes[hashtag_nodes['Modularity Class'] == 1]['Id'])
# for i in range(len(hashtags_list)):
#     hashtags_list[i] = hashtags_list[i][1:]

# hashtags_list = ['covid19',
#                  'gaslighting',
#                  'plandemic',
#                  'freedom',
#                  'vaccines',
#                  'truthmatters',
#                  'covid',
#                  'nonprofit',
#                  'antivaxxers',
#                  'community',
#                  'education',
#                  'science ']

# hashtags_list = ['climatechange',
#                  'climatecrisis',
#                  'suistainable',
#                  'savetheplanet',
#                  'zerowaste',
#                  'climatecrisis',
#                  'climate']

# hashtags_list = ['domesticviolence',
#                  'domesticabuse',
#                  'coercivecontrol',
#                  'abuse',
#                  'guncontrol',
#                  '2a',
#                  'guns',
#                  'gunsense',
#                  'redflaglaws']

# hashtags_list = ['mentalhealth',
#                  'health',
#                  'feminism',
#                  'toxic',
#                  'mentalhealthmatters',
#                  'selfcare',
#                  'metoo',
#                  'facts',
#                  'girls',
#                  'motivation',
#                  'mentalhealthawereness',
#                  'selflove',
#                  'depression',
#                  'run',
#                  'meme',
#                  'memes',
#                  'blacktwitter',
#                  'life',
#                  'selfguarding',
#                  'swipeleft',
#                  'tinder',
#                  'wtf',
#                  'mindfulness',
#                  'awereness',
#                  'cannabiscommunity',
#                  'cbd']

# hashtags_list = ['dating',
#                  'relationships',
#                  'relationshipadvice',
#                  'reallife',
#                  'relatable',
#                  'flag',
#                  'stayway',
#                  'funny',
#                  'humor',
#                  'datingadvice',
#                  'datingtips',
#                  'comedy',
#                  'women',
#                  'men',
#                  'datingredflags',
#                  'alert',
#                  'lgtb',
#                  'horoscope']

hashtags_list = ['business',
                 'digitalmarketing',
                 'fraud',
                 'trading',
                 'money',
                 'advice',
                 'facebook',
                 'twitter',
                 'trend',
                 'linkedin',
                 'instagram',
                 'entrepreneur']

In [96]:
tweets_comm = tweets_filtered.copy().reset_index()
# Procedure to filtrate only tweets with hashtag inside the community
tweets_comm = tweets_filtered.copy().reset_index()
idx_list = []
for c in hashtags_list:
    idx_list.append(list(tweets_comm[tweets_comm['hashtags'].apply(lambda x: c in x)].index))
idx = np.unique(list(chain(*idx_list)))
tweets_comm = tweets_comm.iloc[list(idx)]

,index,Datetime,Tweet Id,Text,Username,Hashtags,clean_text,hashtags
55,989,2021-11-28 01:26:24+00:00,1464767600597082117,5 Red flags to watch out for at the beginning ...,RuthlessCI,"[Relationship, RedFlag, Love, Advice, FairyTale]",red flag watch beginning relationship read ful...,"[relationship, redflag, love, advice, fairytale]"
207,1141,2021-11-22 15:30:18+00:00,1462805647754514432,#BadData is no good. We at Merrill Research id...,ResearchMerrill,"[BadData, data, business, RedFlag]",good research identify bad evaluate individual...,"[baddata, data, business, redflag]"
238,1172,2021-11-21 01:26:22+00:00,1462230876683710467,Nobody's perfect.\n\nRead the full article: 5 ...,RuthlessCI,"[Relationship, RedFlag, Love, Advice, FairyTale]",nobody perfect read full article reason prince...,"[relationship, redflag, love, advice, fairytale]"
275,1209,2021-11-19 17:30:00+00:00,1461748607120461830,🚩Be on the lookout for #accountingfraud red fl...,SurgentAFE,"[accountingfraud, Surgent, fraud, financialsta...",lookout red flag offer content assess risk aud...,"[accountingfraud, surgent, fraud, financialsta..."
348,1282,2021-11-17 15:00:41+00:00,1460986255013990405,Don't do it to fit in do it because your finan...,hjmmanagement,"[hjmman, hjmmanagement, redflag, trend, budget...",fit finance say,"[hjmman, hjmmanagement, redflag, trend, budget..."
...,...,...,...,...,...,...,...,...
4727,5661,2021-10-05 14:28:19+00:00,1445395431136059396,"Beware The Facebook ""Whistleblower""! She Is A ...",PixelOkie,"[DeleteFacebook, Facebook, facebookwhistleblow...",beware fraud plant demand censorship,"[deletefacebook, facebook, facebookwhistleblow..."
4882,5816,2021-09-29 04:28:33+00:00,1443070168389734400,Dating #redflag🚩: someone who doesn't support ...,Cafe24_ph,"[redflag, business, Cafe24, onlinebusiness]",date someone support support register,"[redflag, business, cafe24, onlinebusiness]"
5265,6199,2021-09-09 05:20:01+00:00,1435835363021438978,"Clients who tell you a job should be ""easy"", ""...",lutrov,"[redflag, webdev, webdesign, business]",client tell job easy simple quick rarely worth,"[redflag, webdev, webdesign, business]"
5271,6205,2021-09-08 17:50:13+00:00,1435661768328564737,@VivintHome on what planet do u send people to...,DavidJKelley,"[redflag, redflag, redflag, Fraud, SecurityRisk]",planet you send people someone house w appoint...,"[redflag, redflag, redflag, fraud, securityrisk]"


### Unique words in community

In [99]:
# Where are we working:
dataframe = tweets_comm.copy()
#initialize an empty dict  
unique_words = {} 
for row in dataframe.clean_text:
    for word in row.split(" "):
        #if the word is encountered for the first time add to dict as key and set its value to 0
        unique_words.setdefault(word,0)
        #increase the value (i.e the count) of the word by 1 every time it is encountered
        unique_words[word] += 1 # get unique words and their frequences
word_nodes = pd.DataFrame.from_dict(unique_words, orient="index")
word_nodes.reset_index(inplace=True)
word_nodes["Label"] = word_nodes["index"]
word_nodes.rename(columns={"index":"Id", 0:"delete"},inplace=True)
word_nodes = word_nodes.drop(columns=['delete']) 

uw_df = pd.DataFrame.from_dict(unique_words, orient='index').reset_index()
uw_df.rename(columns = {'index':'Word', 0:'Count'}, inplace=True)
uw_df.sort_values(by=['Count'], ascending=False, inplace=True)
uw = unique_words.keys()

### Words network

In [105]:
h = 0
network = {} 
network_key = 0
for index, row in dataframe.iterrows():
    #hashtags extracted from Tweet do not have the # sign in front of them but we will add it to differentiate hashtags from words
    combined_list = [word for word in str.split(row["clean_text"], " ") if word in uw]
    #itertool product creates Cartesian product of each element in the combined list (hastag + words, both UNIQUE)
    for pair in itertools.product(combined_list, combined_list):
        #exclude self-loops and count each pair only once because our graph is undirected and we do not take self-loops into account
        if pair[0]!=pair[1] and not(pair[::-1] in network):
            network.setdefault(pair, 0)
            network[pair] += 1 

#network = {key: value for (key, value) in network.items() if value >= h} #filter the network
network_df = pd.DataFrame.from_dict(network, orient="index")

network_df.reset_index(inplace=True) 
network_df.columns = ["pair","weight"] # undirected graph 

#to get weighted graph we need a list of 3-element tuplels (u,v,w) where u and v are nodes and w is a number representing weight
up_weighted = []
for edge in network:
    #we can filter edges by weight by uncommenting the next line and setting desired weight threshold
    #if(network[edge])>1: 
    up_weighted.append((edge[0],edge[1],network[edge]))

G = nx.Graph()
G.add_weighted_edges_from(up_weighted)

to_remove = list(set(list(word_nodes['Id'])) ^ set(list(G.nodes())))
for i in range(len(to_remove)):
    word_nodes = word_nodes.drop(word_nodes[word_nodes['Id'] == to_remove[i]].index)

Node_list and Edge_list saving procedure for Gephi visualization:

In [37]:
path = "./DATA_COMM/"
comm = '12'

# SAVE NODE LIST
word_nodes.to_csv(path + "#nodelist_community"+comm+".csv", index=False) 

# We need to insert a column called "Modularity Class" and label
file = pd.read_csv(path + "#nodelist_community"+comm+".csv", sep = ",", header = 0, names = ['Id','Modularity Class'])
file.insert(loc = 1, column = 'Label', value = file['Id'])
file.to_csv(path + "#nodelist_community"+comm+".csv", index=False)

nx.write_weighted_edgelist(G, path + "#edgelist_community"+comm+".csv")

# We need to insert source, target, weight and type of edge
file = pd.read_csv(path + "#edgelist_community"+comm+".csv", sep = " ", header = 0, names = ['Source','Target','Weight'])
file.insert(loc = 2, column = 'Type', value = ['undirected']*len(file))
file.to_csv(path + "#edgelist_community"+comm+".csv", index=False)